<a href="https://colab.research.google.com/github/DinurakshanRavichandran/Visio-Glance/blob/Pre-Processed-Datasets-NLP/drusenPPFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE
import re

# Load the dataset
file_path = '/content/drive/MyDrive/DSGP PROJECT 29/DATASETS/Synthetic_Drusen_Dataset.csv'
df = pd.read_csv(file_path)

# Handle Missing Values
# Assuming the data might have missing values, we replace them with the most frequent category in each column
for column in df.columns:
    most_frequent = df[column].mode()[0]
    df[column].fillna(most_frequent, inplace=True)

# Feature Engineering
# As an example, create a new feature based on Age and BMI interaction
df['Age_BMI'] = df['Age'] * df['BMI']

# Text Processing
# Simplifying text processing for 'Visual Symptoms' without using advanced libraries
def clean_text(text):
    text = text.lower()  # convert to lowercase
    text = re.sub(r'[^a-z\s]', '', text)  # remove non-alphabetic characters
    words = text.split()  # simple tokenization
    return ' '.join(words)  # join words back to string

df['Visual Symptoms'] = df['Visual Symptoms'].apply(clean_text)

# Encode Categorical Data
le = LabelEncoder()
df['Smoking Status'] = le.fit_transform(df['Smoking Status'])
df['Visual Symptoms'] = le.fit_transform(df['Visual Symptoms'])

# Normalize/Scale Numerical Data
scaler = StandardScaler()
numerical_features = ['Age', 'BMI', 'Blood Pressure', 'Cholesterol Levels', 'Age_BMI']
df[numerical_features] = scaler.fit_transform(df[numerical_features])

# Handle Class Imbalance
smote = SMOTE(random_state=42)
features = df.drop('Diagnosis', axis=1)
target = df['Diagnosis']
features_smote, target_smote = smote.fit_resample(features, target)

# Reconstruct the DataFrame with resampled data
df_smote = pd.DataFrame(features_smote, columns=features.columns)
df_smote['Diagnosis'] = target_smote

# Save Preprocessed Data
preprocessed_file_path = '/content/drive/MyDrive/DSGP PROJECT 29/FINAL MODEL/Preprocessed_Drusen_Dataset.csv'
df_smote.to_csv(preprocessed_file_path, index=False)

preprocessed_file_path


<ipython-input-1-b4a4d659fa47>:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(most_frequent, inplace=True)
<ipython-input-1-b4a4d659fa47>:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 

'/content/drive/MyDrive/DSGP PROJECT 29/FINAL MODEL/Preprocessed_Drusen_Dataset.csv'